In [ ]:
import os, subprocess, json, textwrap, pathlib
repo = pathlib.Path('/workspaces/prime-detail')
print('repo exists', repo.exists())
print(subprocess.run(['git','-C',str(repo),'status','--short'], capture_output=True, text=True).stdout)